**Problem Statement**

Customer is a mobile manufacturer based in the US, who entered the market three years ago. As a new entrant in the sector, they want to understand their competitors and preferences of their users so that they can design their strategies accordingly.They want to tweak the marketing strategies to add more value to their brand, provide features to customers that add the most value, and close the demand-supply gap. Their objective is to increase the market share as well as the brand value.

**Solution:**

The solution has been divided into two parts.

Part 1: Deriving the business insights that are useful for product development and marketing.

Part 2: Creating a sentiment classification engine.

**Part 1: Deriving the business insights that are useful for product development and marketing.**

**Uploading the data from google drive path**

In [ ]:
# Importing the drive to upload the dataset
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


__Importing the required Libraries__

In [ ]:
# For Dataset Operation
import pandas as pd
import numpy as np
pd.set_option('max_colwidth',100)

# For Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# For Text Analytics
import nltk
from nltk import FreqDist
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# For Matrix and Model evaluation
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import precision_recall_curve


### The following data sets are available
1.   meta data of type .json
2.   phone data of type .csv

The meta data holds information from sellers point of view and the phone data holds user information.
We begin by unzipping the meta data file and extracting its content to a list



# Reading the  data



In [ ]:
# Reading the meta data
# importing libraries

import gzip
import shutil

#Path to the meta data zip file. 'capstone' is the folder name under 'My Drive'
path1 = '/content/drive/My Drive/capstone/meta_Cell_Phones_and_Accessories.json.gz'


# Path to meta data .json file
path2 = '/content/drive/My Drive/capstone/meta_Cell_Phones_and_Accessories.json'


# Unzipping the meta data file
with gzip.open(path1, 'rb') as f_in:
    with open(path2, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [ ]:
# Reading the unzipped meta data into a Python list. The result will be a list of dictionaries.
import json

# Empty list to store the dictonaries
phonemetadata = []

# Reading the dictionaries in the json file and appending it to the list phonemetadata[]
with open(path2, 'r') as f:
    for line in f:
        phonemetadata.append(json.loads(line))

In [ ]:
# Getting the number of entries in the phonemetadata list
len(phonemetadata)

590071

In [ ]:
#converting the list phonemetadata into a data frame
df_meta=pd.DataFrame(phonemetadata)

In [ ]:
# Saving the CSV file for Meta data.
df_meta.to_csv('/content/drive/MyDrive/capstone/meta1.csv',escapechar='\\')

In [ ]:
#Reading the .csv file of the phone data into a dataframe

df_phonedata = pd.read_csv('/content/drive/My Drive/capstone/Cell_Phones_and_Accessories_5.csv')

<ipython-input-10-240faa60e68d>:3: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_phonedata = pd.read_csv('/content/drive/My Drive/capstone/Cell_Phones_and_Accessories_5.csv')


__Step 1: Data Understanding and  Preparation__

In [ ]:
df_phonedata.head(5)

,overall,verified,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,review_sentiment
0,5.0,True,A24E3SXTC62LJI,7508492919,{'Color:': ' Bling'},Claudia Valdivia,Looks even better in person. Be careful to not drop your phone so often because the rhinestones ...,Can't stop won't stop looking at it,1407110400,NaN,NaN,POSITIVE
1,5.0,True,A269FLZCB4GIPV,7508492919,NaN,sarah ponce,When you don't want to spend a whole lot of cash but want a great deal...this is the shop to buy...,1,1392163200,NaN,NaN,POSITIVE
2,3.0,True,AB6CHQWHZW4TV,7508492919,NaN,Kai,"so the case came on time, i love the design. I'm actually missing 2 studs but nothing too notice...",Its okay,1391817600,NaN,NaN,NEGATIVE
3,2.0,True,A1M117A53LEI8,7508492919,NaN,Sharon Williams,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY WERE OKAY WITH IT. JUST NOT WHAT I EXPECTED.,CASE,1391472000,NaN,NaN,POSITIVE
4,4.0,True,A272DUT8M88ZS8,7508492919,NaN,Bella Rodriguez,"I liked it because it was cute, but the studs fall off easily and to protect a phone this would ...",Cute!,1391385600,NaN,NaN,POSITIVE


In [ ]:
df_phonedata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128437 entries, 0 to 1128436
Data columns (total 12 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   overall           1128437 non-null  float64
 1   verified          1128437 non-null  bool   
 2   reviewerID        1128437 non-null  object 
 3   asin              1128437 non-null  object 
 4   style             605241 non-null   object 
 5   reviewerName      1128235 non-null  object 
 6   reviewText        1127607 non-null  object 
 7   summary           1127898 non-null  object 
 8   unixReviewTime    1128437 non-null  int64  
 9   vote              92034 non-null    object 
 10  image             27107 non-null    object 
 11  review_sentiment  1128437 non-null  object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 95.8+ MB


# Data pre-processing



In [ ]:
# Converting unix review time to date-time format

#Transforming unixReview time to date time format
from datetime import datetime, timedelta
df_phonedata['Date&Time'] = df_phonedata['unixReviewTime'].apply(lambda d: (datetime.fromtimestamp(d) - timedelta(hours=2)).strftime('%Y-%m-%d'))


In [ ]:
# Dropping the redundant column
df_phonedata.drop(['unixReviewTime'], axis = 1, inplace = True)

In [ ]:
df_phonedata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128437 entries, 0 to 1128436
Data columns (total 12 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   overall           1128437 non-null  float64
 1   verified          1128437 non-null  bool   
 2   reviewerID        1128437 non-null  object 
 3   asin              1128437 non-null  object 
 4   style             605241 non-null   object 
 5   reviewerName      1128235 non-null  object 
 6   reviewText        1127607 non-null  object 
 7   summary           1127898 non-null  object 
 8   vote              92034 non-null    object 
 9   image             27107 non-null    object 
 10  review_sentiment  1128437 non-null  object 
 11  Date&Time         1128437 non-null  object 
dtypes: bool(1), float64(1), object(10)
memory usage: 95.8+ MB


##### EDA for Phone date

##### Checking Null values

In [ ]:
df_phonedata.isnull().mean()*100

overall              0.000000
verified             0.000000
reviewerID           0.000000
asin                 0.000000
style               46.364662
reviewerName         0.017901
reviewText           0.073553
summary              0.047765
vote                91.844117
image               97.597828
review_sentiment     0.000000
Date&Time            0.000000
dtype: float64

Dropping the rows containing null values of Style column.

In [ ]:
df_phonedata=df_phonedata[~df_phonedata['style'].isnull()]

Dropping the column with higher null values.

In [ ]:
df_phonedata.drop(['vote','image'], axis = 1, inplace = True)

In [ ]:
df_phonedata.head(3)

,overall,verified,reviewerID,asin,style,reviewerName,reviewText,summary,review_sentiment,Date&Time
0,5.0,True,A24E3SXTC62LJI,7508492919,{'Color:': ' Bling'},Claudia Valdivia,Looks even better in person. Be careful to not drop your phone so often because the rhinestones ...,Can't stop won't stop looking at it,POSITIVE,2014-08-03
226,5.0,True,A1TNHWAM2GJ08G,961301375X,{'Color:': ' White'},P. Latella,this charger is amazing. Fantastic long cord and it charges the phone beautifully. I bought anot...,Perfect works great,POSITIVE,2014-02-10
227,5.0,True,AXGPQOWI0CF39,961301375X,{'Color:': ' White'},TimB.,I am very satisfied with this product! It came in the appropriate packaging. It was also a very ...,Very Nice Looking Item!,POSITIVE,2014-01-31


In [ ]:
df_phonedata.shape

(605241, 10)

In [ ]:
df_phonedata.isnull().mean()*100

overall             0.000000
verified            0.000000
reviewerID          0.000000
asin                0.000000
style               0.000000
reviewerName        0.018009
reviewText          0.078151
summary             0.054689
review_sentiment    0.000000
Date&Time           0.000000
dtype: float64

In [ ]:
# Dropping the rows with null counts of review text
df_phonedata = df_phonedata[df_phonedata['reviewText'].notna()]

In [ ]:
df_phonedata.isnull().mean()*100

overall             0.000000
verified            0.000000
reviewerID          0.000000
asin                0.000000
style               0.000000
reviewerName        0.018023
reviewText          0.000000
summary             0.048614
review_sentiment    0.000000
Date&Time           0.000000
dtype: float64

In [ ]:
# Dropping the rows with null counts of summary
df_phonedata = df_phonedata[df_phonedata['summary'].notna()]

In [ ]:
df_phonedata.isnull().mean()*100

overall             0.000000
verified            0.000000
reviewerID          0.000000
asin                0.000000
style               0.000000
reviewerName        0.017867
reviewText          0.000000
summary             0.000000
review_sentiment    0.000000
Date&Time           0.000000
dtype: float64

##### Null values of all columns are very  minimal now.

In [ ]:
#reading meta1.csv file
df_meta=pd.read_csv('/content/drive/MyDrive/capstone/meta1.csv')

<ipython-input-20-b1a75eeb77d0>:2: DtypeWarning: Columns (4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_meta=pd.read_csv('/content/drive/MyDrive/capstone/meta1.csv')


In [ ]:
df_meta.head(3)

,Unnamed: 0,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin
0,0,"['Cell Phones & Accessories', 'Accessories', 'Screen Protectors']",NaN,[],NaN,Trollies Sing a Long Party VHS,[],[],NaN,NaN,[],"557,379 in Movies & TV (",[],{'ASIN: ': '6303195164'},Movies & TV,NaN,NaN,NaN,6303195164
1,1,"['Cell Phones & Accessories', 'Accessories', 'Car Accessories', 'Car Cradles & Mounts']",NaN,[],NaN,La Pura VHS,[],[],NaN,Maribel Guardia,[],"1,024,449 in Movies & TV (",[],{'ASIN: ': '630524412X'},Movies & TV,NaN,NaN,NaN,630524412X
2,2,"['Cell Phones & Accessories', 'Cases, Holsters & Sleeves']",NaN,"['New items to give you a new good mood, welcome to you have a happy shopping trip.']",NaN,Leopard Thanksgiving Easter PC Black Case for Masterpiece Limited Design iphone 6 by Cases &amp;...,[],[],NaN,NaN,[],"5,315,017 in Books (",[],"{'Publisher:': 'juduiou', 'ASIN:': '6310200240', '\\n Product Dimensions: \\n ': '11.5 x 1...",Books,NaN,NaN,NaN,6310200240


In [ ]:
df_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590071 entries, 0 to 590070
Data columns (total 19 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Unnamed: 0    590071 non-null  int64 
 1   category      590071 non-null  object
 2   tech1         6585 non-null    object
 3   description   590071 non-null  object
 4   fit           43 non-null      object
 5   title         590062 non-null  object
 6   also_buy      590071 non-null  object
 7   image         590071 non-null  object
 8   tech2         231 non-null     object
 9   brand         577271 non-null  object
 10  feature       590071 non-null  object
 11  rank          590071 non-null  object
 12  also_view     590071 non-null  object
 13  details       590071 non-null  object
 14  main_cat      586685 non-null  object
 15  similar_item  50931 non-null   object
 16  date          45880 non-null   object
 17  price         127564 non-null  object
 18  asin          590071 non

In [ ]:
df_meta.isnull().mean()*100

Unnamed: 0       0.000000
category         0.000000
tech1           98.884033
description      0.000000
fit             99.992713
title            0.001525
also_buy         0.000000
image            0.000000
tech2           99.960852
brand            2.169230
feature          0.000000
rank             0.000000
also_view        0.000000
details          0.000000
main_cat         0.573829
similar_item    91.368666
date            92.224664
price           78.381585
asin             0.000000
dtype: float64

In [ ]:
#dropping duplicates in ASIN

df_meta.drop_duplicates(subset='asin',inplace=True)

In [ ]:
# dropping the columns with higher percentage null values, except Price column
df_meta.drop(['tech1','tech2','fit','date','similar_item'], axis = 1, inplace = True)

In [ ]:
df_meta.isnull().sum()

Unnamed: 0          0
category            0
description         0
title               9
also_buy            0
image               0
brand           12774
feature             0
rank                0
also_view           0
details             0
main_cat         3386
price          461899
asin                0
dtype: int64

In [ ]:
df_meta.price.value_counts()

price
$7.99      6290
$9.99      6203
$6.99      5131
$8.99      4383
$14.99     3728
           ... 
$65.24        1
$100.60       1
$23.30        1
$38.29        1
$75.24        1
Name: count, Length: 4668, dtype: int64

In [ ]:
# replace $ to none and convert price to float
df_meta["price"]= df_meta["price"].str.replace("$",'')

In [ ]:
df_meta.price.value_counts()

price
7.99      6290
9.99      6203
6.99      5131
8.99      4383
14.99     3728
          ... 
65.24        1
100.60       1
23.30        1
38.29        1
75.24        1
Name: count, Length: 4668, dtype: int64

In [ ]:
# replace "," to none
df_meta["price"]= df_meta["price"].str.replace(",",'')

In [ ]:
# converting price column from object dtype to numeric
df_meta['price']= pd.to_numeric(df_meta['price'], errors='coerce')

In [ ]:
# checking the overall null values in price column
df_meta["price"].isnull().sum()

464863

In [ ]:
# checking the brand wise null value
df_meta[df_meta["price"].isnull()]['brand'].value_counts()

brand
Generic                                9696
EMPIRE                                 4648
MyBat                                  4466
Unknown                                4142
Samsung                                3198
                                       ... 
Pyra                                      1
Lucdan                                    1
CIDEROS                                   1
TPE Electronics Technology Co.,Ltd.       1
HBER                                      1
Name: count, Length: 39723, dtype: int64

In [ ]:
# imputing null price with their brand's mean price
df_meta['price'] = df_meta.groupby('brand')['price'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
df_meta["price"].isnull().sum()

167235

In [ ]:
df_meta.isnull().sum()

Unnamed: 0          0
category            0
description         0
title               9
also_buy            0
image               0
brand           12774
feature             0
rank                0
also_view           0
details             0
main_cat         3386
price          167235
asin                0
dtype: int64

In [ ]:
df_meta=df_meta[~df_meta['main_cat'].isna()]

In [ ]:
df_meta.isnull().sum()

Unnamed: 0          0
category            0
description         0
title               7
also_buy            0
image               0
brand           11547
feature             0
rank                0
also_view           0
details             0
main_cat            0
price          165418
asin                0
dtype: int64

In [ ]:
# Rest all missing values replaced by mean price
df_meta['price'] = df_meta['price'].fillna(df_meta['price'].mean())

In [ ]:
df_meta=df_meta[~df_meta['brand'].isna()]

In [ ]:
df_meta.isnull().sum()

Unnamed: 0     0
category       0
description    0
title          7
also_buy       0
image          0
brand          0
feature        0
rank           0
also_view      0
details        0
main_cat       0
price          0
asin           0
dtype: int64

__There are no major null values available.__

##### Extracting rank

In [ ]:
df_meta['rank'].value_counts()

rank
[]                                                                                                                                                                                                                                                                57832
['>#1,135 in Cell Phones & Accessories (See Top 100 in Cell Phones & Accessories)', '>#168 in Cell Phones & Accessories > Cell Phone Accessories > Screen Protectors', '>#1,134 in Electronics > Cell Phones & Accessories']                                         77
['>#189,670 in Cell Phones & Accessories (See Top 100 in Cell Phones & Accessories)', '>#13,064 in Cell Phones & Accessories > Cell Phone Accessories > Screen Protectors', '>#189,241 in Electronics > Cell Phones & Accessories']                                  44
['>#139,758 in Kitchen & Dining (See Top 100 in Kitchen & Dining)', '>#56,050 in Electronics > Cell Phones & Accessories > Cases, Holsters & Clips > Basic Cases']                                         

In [ ]:
# extracting rank in cellphones and acessories
df_meta["new_rank"]=df_meta["rank"].str.extract(pat='([^>#](\d{0,3},)?(\d{3},)?\d{0,3}\s*in Cell Phones & Accessories)')[0]

In [ ]:
df_meta["new_rank"]=df_meta["new_rank"].str.replace(">#","").str.replace("in Cell Phones & Accessories","").str.replace(",","")

In [ ]:
# converting new rank column to integer
df_meta["new_rank"] = df_meta["new_rank"].astype("float")

In [ ]:
# dropping rank column as we got new_rank column
df_meta.drop(["rank"],axis=1, inplace=True)

In [ ]:
df_meta.shape

(574423, 14)

In [ ]:
# writing meta data csv file to external location
df_meta.to_csv('/content/drive/MyDrive/capstone/meta_data.csv')

#### __Merging both dataframes__

In [ ]:
# Merging two dataframes
df_merged = pd.merge(df_phonedata, df_meta, how = 'inner', on = ['asin'])

In [ ]:
df_merged.head(3)

,overall,verified,reviewerID,asin,style,reviewerName,reviewText,summary,review_sentiment,Date&Time,...,title,also_buy,image,brand,feature,also_view,details,main_cat,price,new_rank
0,5.0,True,A24E3SXTC62LJI,7508492919,{'Color:': ' Bling'},Claudia Valdivia,Looks even better in person. Be careful to not drop your phone so often because the rhinestones ...,Can't stop won't stop looking at it,POSITIVE,2014-08-03,...,MinisDesign 3d Bling Crystal Bow Transparent Case for Apple Iphone 4 and 4s-Pink,[],"['https://images-na.ssl-images-amazon.com/images/I/41Wol0yPcxL._SX38_SY50_CR,0,0,38,50_.jpg']",ELEGANI,"['Fashionable with unique 3D butterfly design for the iPhone 5 from others', 'Precise cut design...",[],{},Cell Phones & Accessories,9.82875,228544.0
1,4.0,True,AX8YW6N99IWPQ,7508492919,{'Color:': ' Bling'},eladia paz,Happy it,Four Stars,POSITIVE,2015-02-14,...,MinisDesign 3d Bling Crystal Bow Transparent Case for Apple Iphone 4 and 4s-Pink,[],"['https://images-na.ssl-images-amazon.com/images/I/41Wol0yPcxL._SX38_SY50_CR,0,0,38,50_.jpg']",ELEGANI,"['Fashionable with unique 3D butterfly design for the iPhone 5 from others', 'Precise cut design...",[],{},Cell Phones & Accessories,9.82875,228544.0
2,3.0,True,A3P9BQ6A7EO0FN,7508492919,{'Color:': ' Bling'},Susan Holt,It was a very cute phone case.,Cute Case,POSITIVE,2015-01-27,...,MinisDesign 3d Bling Crystal Bow Transparent Case for Apple Iphone 4 and 4s-Pink,[],"['https://images-na.ssl-images-amazon.com/images/I/41Wol0yPcxL._SX38_SY50_CR,0,0,38,50_.jpg']",ELEGANI,"['Fashionable with unique 3D butterfly design for the iPhone 5 from others', 'Precise cut design...",[],{},Cell Phones & Accessories,9.82875,228544.0


In [ ]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 603879 entries, 0 to 603878
Data columns (total 23 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   overall           603879 non-null  float64
 1   verified          603879 non-null  bool   
 2   reviewerID        603879 non-null  object 
 3   asin              603879 non-null  object 
 4   style             603879 non-null  object 
 5   reviewerName      603771 non-null  object 
 6   reviewText        603879 non-null  object 
 7   summary           603879 non-null  object 
 8   review_sentiment  603879 non-null  object 
 9   Date&Time         603879 non-null  object 
 10  Unnamed: 0        603879 non-null  int64  
 11  category          603879 non-null  object 
 12  description       603879 non-null  object 
 13  title             603879 non-null  object 
 14  also_buy          603879 non-null  object 
 15  image             603879 non-null  object 
 16  brand             60

In [ ]:
# Creating merged csv file
df_merged.to_csv('/content/drive/MyDrive/capstone/merged.csv')

In [ ]:
#pip install xlsxwriter

In [ ]:
# Converting to xlsx format and saving
#df_merged1.to_excel(r"/content/drive/My Drive/capstone/merged_excelfile.xlsx", index=False, header=True, engine= 'xlsxwriter')
